In [1]:
from selenium.webdriver.common.by import By
from selenium import webdriver
import time
import pandas as pd

##### You should specify the path to the chromedriver that is located in your computer and assign it to the path variable below.

In [2]:
url = 'https://en.bidfax.info/'
path = '/Users/areg/.wdm/drivers/chromedriver/mac64/108.0.5359/chromedriver'
# You should specify the path of the chromedriver in the brackets 
# that are in front of path variable above
driver = webdriver.Chrome(path)
driver.get(url)

/var/folders/7_/hrpf2wd52k91vfg1qd2m7ch80000gn/T/ipykernel_22936/3681283350.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


#### After calling select_make() function it will ask you to input a make of a car and then it will select it. If that make will not exit it will tell you to put the existing one.

In [3]:
def select_make():
    dropdown = driver.find_elements(By.XPATH, '//span[@class = "drop-down"]' )
    make = str(input('Please write the make you want to select:  '))
    def search_for_make(make):
        try:
            dropdown[0].click()
        except:
            pass
        try:
            make_ = driver.find_element(By.LINK_TEXT, make)
            make_.click()
            print(f"{make} make is selected successfully!")
        except:
            print(f'{make} make does not exist in this list. Please try again.')
            return select_make()
    search_for_make(make)

#### After calling select_model() fucntion it will ask you to input a model of a car and then it will select it. If that make will not exit it will tell you to put the existing one.

In [4]:
def select_model():
    dropdown = driver.find_elements(By.XPATH, '//span[@class = "drop-down"]' )
    model = str(input('Please write the model you want to select:  '))
    def search_for_model(model):
        try:
            dropdown[1].click()
        except:
            pass
        try:
            model_ = driver.find_element(By.LINK_TEXT, model)
            model_.click()
            print(f"{model} model is selected successfully!")
        except:
            print(f'{model} model does not exist in this list. Please try again.')
            return select_model()
    search_for_model(model)  

#### After calling select_year_start() function below it will ask you to input the starting year of a car you want to search for and then it will select it. If that make will not exit it will tell you to put the existing one. Call the select_year_start function after calling select_make() and select_model() functions

In [5]:
def select_year_start():
    def scrape_years():
        global years
        years_dropdown = driver.find_element(By.XPATH, '//div[@class = "filter_right"]')
        years = years_dropdown.find_elements(By.XPATH, ('./select'))
    scrape_years()
    start = str(input("Type in the starting year of the car:  "))
    def year_start(start):   
        year_start = years[0]
        try:
            year_start = year_start.find_elements(By.XPATH, ("./option"))
            for year in year_start:
                if year.text == start:
                    print(f"{start} year is selected successfully!")
                    return year.click()
            raise print(f"{start} year does not exist. Please try again.")
        except:
            return select_year_start()
    year_start(start)

#### After calling select_year_to() function it will ask you to input the ending year of a car you want to search for and then it will select it. If that make will not exit it will tell you to put the existing one.  Call the select_year_start function after calling select_make() and select_model() functions

In [6]:
def select_year_to():
    def scrape_years():
        global years
        years_dropdown = driver.find_element(By.XPATH, '//div[@class = "filter_right"]')
        years = years_dropdown.find_elements(By.XPATH, ('./select'))    
    scrape_years()
    to = str(input("Type in the ending year of the car:  "))
    def year_end(year_to):   
        year_to = years[1]
        try:
            year_to = year_to.find_elements(By.XPATH, ("./option"))
            for year in year_to:
                if year.text == to:
                    print(f"{to} year is selected successfully!")
                    return year.click()
            raise print(f"{to} year does not exist. Please try again.")
        except:
            return select_year_to()
    year_end(to)
    

In [7]:
def go_in_car():
    cars = driver.find_elements(By.XPATH,'//img[@src = "https://bidfax.info/templates/ru/dleimages/sold2.png"]')
    
    for i in range(len(cars)):
        a = (driver.find_elements(By.XPATH, '//img[@src = "https://bidfax.info/templates/ru/dleimages/sold2.png"]/parent::p/preceding-sibling::a'))
        a[i].click()
        time.sleep(1)
        
        driver.current_window_handle
        driver.back()
        time.sleep(2)
go_in_car()
    



In [10]:
select_make()
select_model()
select_year_start()
select_year_to()


Lexus make is selected successfully!
Es model is selected successfully!
2017 year is selected successfully!
2020 year is selected successfully!


In [11]:
go_in_car()

IndexError: list index out of range